# Bitemporale Graphen
Bitemporale Graphen sind eine spezielle Art von Graphen, die neben Beziehungen zwischen Knoten auch mehrere zeitliche Aspekte berücksichtigen.

In [ ]:
from datetime import date
import networkx as nx

## Inhaltsverzeichnis
- [Einführung und Definitionen](#Einführung-und-Definitionen)
- [Umsetzung mit NetworkX](#Umsetzung-mit-NetworkX)
- [Visualisierung](#Visualisierung)
- [Zusammenfassung](#Zusammenfassung)

## Einführung und Definitionen
Nach einem Umzug muss innerhalb von 14 Tagen eine Ummeldung beim Einwohnermeldeamt erfolgen. Wer also am 1. Juni durch den Einzug einen Fakt schafft, kann trotzdem fristgerecht am 12. Juni in der Datenbank des Einwohnermeldeamtes eine neue Adresse erhalten. Wird nur eine der beiden Angaben gespeichert, geht entweder die historische Realität (Umzugsdatum) oder die Historie im System (Verbuchungsdatum) verloren.

In einem bitemporalen Graphen werden deshalb jedem Knoten und jeder Kante zwei Zeitstempel bzw. Zeitdimensionen (deshalb **bi**temporal) hinzugefügt:
- Die **Gültigkeitszeit** (*Valid Time*) gibt an, wann ein durch den Graph repräsentierter Fakt in der realen Welt gültig war.
- Die **Transaktionszeit** (*Transaction Time*) gibt an, wann ein durch den Graph repräsentierter Fakt gespeichert oder geändert wurde.

Umgesetzt werden diese beiden zusätzlichen Attribute durch zwei Funktionen, die ähnlich wie das Gewicht modelliert werden:
- $from_v, to_v : (V \cup E) \rightarrow \Omega_{v}$ bilden einen Knoten oder eine Kante auf eine Start- und eine Endzeit aus der Menge der **Gültigkeit**szeiten ab.
- $from_t, to_v : (V \cup E) \rightarrow \Omega_{t}$ bildet einen Knoten oder eine Kante auf eine Start- und eine Endzeit aus der Menge der **Transaktion**szeiten ab.

Zusätzlich werden einige Anforderungen formuliert:
1. $\forall e \in V \cup E : from_v(e) < to_v(e)$
2. $\forall e \in V \cup E : from_t(e) < to_t(e)$
3. $\forall e = (u, v) \in E : from_v(e) \geq max(from_v(u), from_v(v)) \land to_v(e) \leq min(to_v(u), to_t(v))$
4. $\forall e = (u, v) \in E : from_t(e) \geq max(from_t(u), from_t(v)) \land to_t(e) \leq min(to_t(u), to_t(v))$

bzw:

1. Der Startzeitpunkt muss für beide Angaben immer vor dem Endzeitpunkt liegen.
2. Die Zeiten für Kanten werden durch die Zeiten der zur Kante gehörenden Knoten beschränkt.

Im Gegensatz zu nicht-temporalen Graphen müssen die Knoten und Kanten außerdem nicht eindeutig sein. Relevant ist nur, dass sie Transaktionszeiten besitzen, die sich nicht überschneiden.

## Umsetzung mit NetworkX
NetworkX bietet keine direkte Unterstützung für bitemporale Graphen. Durch Hinzufügen separater Attribute zu Knoten und Kanten und Anpassung der verwendeten Algorithmen, kann allerdings diese Funktionalität trotzdem modelliert werden.

In [ ]:
G = nx.DiGraph()

G.add_edge('Person', 'Wohnung A',
           valid_time=(date(2025, 1, 1), date(2025, 5, 31)),
           transaction_time=(date(2025, 1, 2), date(2025, 1, 2)))

G.add_edge('Person', 'Wohnung B',
           valid_time=(date(2025, 6, 1), None),
           transaction_time=(date(2025, 6, 12), date(2025, 6, 12)))

for u, v, data in G.edges(data=True):
    print(f'{u} -> {v}: {data}')

Für die Abfrage der zur einem spezifischen Zeitpunkt gültigen Informationen, müssen dann die Algorithmen angepasst werden:

In [ ]:
ts = date(2025, 5, 15)

for u, v, data in G.edges(data=True):
    transaction_start, transaction_end = data['transaction_time']
    if transaction_end < ts:
        print(f'{u} -> {v}: {data}')

### Umsetzung in SQL:2011
SQL:2011 bietet Unterstützung für temporale Daten. Dazu wurden unter anderem Zeitintervalle (`PERIOD FOR`), temporale Primärschlüssel (`WITHOUT OVERLAPS`) eingeführt und automatische Versionierung (`WITH SYSTEM VERSIONING`) eingeführt.

```sql
CREATE TABLE Wohnsitz (
    Person            VARCHAR,
    Wohnung           VARCHAR,
    valid_from        DATE,
    valid_to          DATE,
    transaction_from  TIMESTAMP GENERATED ALWAYS AS ROW START,
    transaction_end   TIMESTAMP GENERATED ALWAYS AS ROW END,
    PERIOD FOR valid_time (valid_from, valid_to),
    PERIOD FOR SYSTEM_TIME (transaction_start, transaction_end),
    PRIMARY KEY (Person, valid_time WITHOUT OVERLAPS)
) WITH SYSTEM VERSIONING
```

In dieser Datenbank lässt sich nun beispielsweise nach gültigen Werten zu einem bestimmten Zeitpunkt (`FOR SYSTEM_TIME AS OF`) suchen.

```sql
SELECT *
FROM Wohnsitz
FOR SYSTEM_TIME AS OF DATE '2025-05-15'
```

Die Unterstützung für temporale Daten ist vom verwendeten Datenbanksystem abhängig. Manche [unterstützen nativ einen Teil](https://mariadb.com/kb/en/system-versioned-tables/) der vorgeschlagenen Definitionen und Abfragemuster, andere verwenden [Plugins zur Erweiterung der Kernfunktionalität](https://wiki.postgresql.org/wiki/Temporal_Extensions).

## Visualisierung
Nicht alle Informationen eines bitemporalen Graphen sind zu jedem beliebigen Zeitpunkt gültig. Die Visualisierung bitemporaler Graphen erfordert daher eine gewisse Dynamik. Dazu können Animationen eingesetzt werden, welche das Durchschreiten der Zeit in einer Art Timeline erlauben, aber auch Entwicklungen können durch Differenzen oder Heatmaps dargestellt werden.

## Zusammenfassung
In diesem Abschnitt wurden bitemporale Graphen in Ansätzen vorgestellt. Sie eignen sich für Modellierungen, in denen zwei zeitliche Aspekte eine Rolle spielen und Analysen zu einem spezifischen Zeitpunkt relevant sind.